<a href="https://colab.research.google.com/github/aryan1429/Big-Data/blob/main/MatrixMultiplication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update -qq
!apt-get install -y openjdk-8-jdk-headless > /dev/null
!wget -q https://downloads.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz
!tar -xzf hadoop-3.3.6.tar.gz
!mv hadoop-3.3.6 /usr/local/hadoop


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [2]:
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["HADOOP_HOME"] = "/usr/local/hadoop"
os.environ["PATH"] += f":{os.environ['HADOOP_HOME']}/bin:{os.environ['HADOOP_HOME']}/sbin"

!hadoop version


Hadoop 3.3.6
Source code repository https://github.com/apache/hadoop.git -r 1be78238728da9266a4f88195058f08fd012bf9c
Compiled by ubuntu on 2023-06-18T08:22Z
Compiled on platform linux-x86_64
Compiled with protoc 3.7.1
From source with checksum 5652179ad55f76cb287d9c633bb53bbd
This command was run using /usr/local/hadoop/share/hadoop/common/hadoop-common-3.3.6.jar


In [3]:
!echo "This is a sample file for Hadoop testing." > sample.txt
!cat sample.txt

This is a sample file for Hadoop testing.


In [4]:
%%writefile matrix_input.txt
A,0,0,1
A,0,1,2
A,0,2,3
A,1,0,4
A,1,1,5
A,1,2,6
B,0,0,7
B,0,1,8
B,1,0,9
B,1,1,10
B,2,0,11
B,2,1,12

Writing matrix_input.txt


In [5]:
%%writefile mapper.py
#!/usr/bin/env python3
import sys
n = 3
p = 2
for line in sys.stdin:
  line = line.strip()
  if not line:
    continue
  name,i,j,val = line.split(',')
  i,j,val = int(i),int(j),float(val)
  if name == 'A':
    for col in range(p):
      print(f"{i},{col}\tA,{j},{val}")
  else:
    for row in range(2):
      print(f"{row},{j}\tB,{i},{val}")

Writing mapper.py


In [12]:
%%writefile reducer.py
#!/usr/bin/env python3
import sys
from collections import defaultdict
current_key = None
A = defaultdict(float)
B = defaultdict(float)
def emit (key,A,B):
  total = 0
  for k in A:
    total += A[k] * B.get(k,0)
  print(f"{key}\t{total}")
for line in sys.stdin:
  line = line.strip()
  if not line:
    continue
  key,val = line.split('\t')
  if current_key and key != current_key:
    emit(current_key,A,B)
    A = defaultdict(float)
    B = defaultdict(float)
  current_key = key
  tag,k,v= val.split(',')
  k,v = int(k),float(v)
  if tag =='A':
    A[k] = v
  else:
    B[k] = v
if current_key:
  emit(current_key,A,B)

Overwriting reducer.py


In [13]:
!chmod +x mapper.py reducer.py

In [14]:
!hdfs dfs -mkdir /matrix

mkdir: `/matrix': File exists


In [15]:
!touch matrix.txt


In [10]:
!hdfs dfs -put matrix.txt /matrix

In [16]:
!hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.3.6.jar \
    -input matrix_input.txt \
    -output output_matrix \
    -mapper mapper.py \
    -reducer reducer.py


2025-11-02 06:06:15,824 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2025-11-02 06:06:15,919 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2025-11-02 06:06:15,919 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2025-11-02 06:06:15,939 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2025-11-02 06:06:15,970 ERROR streaming.StreamJob: Error Launching job : Output directory file:/content/output_matrix already exists
Streaming Command Failed!


In [17]:
!rm -r /content/output_matrix


In [18]:
!hadoop jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-*.jar \
-input /content/matrix_input.txt \
-output /content/output_matrix \
-mapper /content/mapper.py \
-reducer /content/reducer.py


2025-11-02 06:07:13,986 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2025-11-02 06:07:14,095 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2025-11-02 06:07:14,095 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2025-11-02 06:07:14,131 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2025-11-02 06:07:14,441 INFO mapred.FileInputFormat: Total input files to process : 1
2025-11-02 06:07:14,470 INFO mapreduce.JobSubmitter: number of splits:1
2025-11-02 06:07:14,706 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1043502234_0001
2025-11-02 06:07:14,706 INFO mapreduce.JobSubmitter: Executing with tokens: []
2025-11-02 06:07:14,965 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
2025-11-02 06:07:14,969 INFO mapred.LocalJobRunner: OutputCommitter set in config null
2025-11-02 06:07:14,969 INFO mapreduce.Job: Running job: job_local1043502234_0001
2025-11

In [19]:
!cat /content/output_matrix/part-00000


0,0	58.0
0,1	64.0
1,0	139.0
1,1	154.0
